In [1]:
import pandas as pd
import numpy as np
import gc
import os
from os import listdir
from os.path import isfile, join
import time

In [2]:
# Arguments
min_length = 5
max_length = 20
source_folder_name = 'min_5_max_20'
project_folder = '/data/workspace/yeqi/projects/RNN4REC/GRU4REC'
data_folder = '/data/workspace/yeqi/projects/RNN4REC/GRU4REC/Data/paths'

train_folder = data_folder + '/training set'
test_folder = data_folder + '/test set'
full_folder = data_folder + '/full data'

pro_data_folder = project_folder + '/Processed Data'
source_folder = pro_data_folder + '/' + source_folder_name

In [3]:
X_train = np.load(source_folder +'/X_train.npy')
X_test = np.load(source_folder +'/X_test.npy')

In [4]:
userid_df = pd.read_csv(source_folder + '/userid_map.csv')
itemid_df = pd.read_csv(source_folder + '/itemid_map.csv')

In [5]:
item_df = pd.DataFrame(itemid_df)
item_df['train_count'] = 0
item_df['test_count'] = 0

for row in X_train:
    for itemid in row[2]:
        item_df['train_count'][itemid] = item_df['train_count'][itemid] + 1
for row in X_test:
    for itemid in row[2]:
        item_df['test_count'][itemid] = item_df['test_count'][itemid] + 1

train_item_count = 0
test_item_count = 0
co_item_count = 0
item_arr = item_df.values
for row in item_arr:
    if row[2] > 0:
        train_item_count = train_item_count + 1
    if row[3] > 0:
        test_item_count = test_item_count + 1
        if row[2] > 0:
            co_item_count = co_item_count + 1

train_item_count, test_item_count, co_item_count

(193077, 24344, 12597)

In [6]:
item_arr = item_df.values

np.mean(np.array(item_df['train_count']))

top_items = []
for i in range(1, len(item_arr)):
    row = item_arr[i]
    if row[2] >= 20 and row[3] > 0:
        top_items.append(list(row))
        
top_items = np.array(top_items)
top_i_df = pd.DataFrame(top_items)

In [8]:
top_items_list = list(top_items[:,0])

In [9]:
len(top_items_list)

3814

In [10]:
itemid_dict = dict([(top_items_list[i], i) for i in range(len(top_items_list))])

In [11]:
# reimport our sliced training data and origin test data
X_train = np.load("/data/workspace/yeqi/projects/RNN4REC/GRU4REC/Processed Data/min_5_max_20/sliced data/X_train.npy")
X_test = np.load("/data/workspace/yeqi/projects/RNN4REC/GRU4REC/Processed Data/min_5_max_20/X_test.npy")

In [12]:
X_train.shape, X_test.shape

((679315, 5), (16152, 5))

In [13]:
new_X_test = []

for row in X_test:
    if row[4] in top_items_list:
        new_row = []
        new_row.append(row[0])
        new_row.append(row[1])
        new_row.append(row[2])
        new_row.append(row[3])
        new_row.append(itemid_dict[row[4]])
        new_X_test.append(new_row)
new_X_test = np.array(new_X_test, dtype = object)

In [14]:
new_X_train = []
for row in X_train:
    if row[4] in top_items_list:
        new_row = []
        new_row.append(row[0])
        new_row.append(row[1])
        new_row.append(row[2])
        new_row.append(row[3])
        new_row.append(itemid_dict[row[4]])
        new_X_train.append(new_row)
new_X_train = np.array(new_X_train, dtype = object)

In [15]:
new_X_train.shape, new_X_test.shape

((124037, 5), (2465, 5))

In [16]:
new_X_train

array([[11445, 52, array([115639, 168292, 168295, 106023]), 5, 739],
       [17479, 7, array([155881,  55112, 123739, 169190]), 5, 2776],
       [8503, 12, array([ 24992, 211561, 194697, 112275]), 5, 149],
       ...,
       [9500, 3,
        array([ 22235,  86016,  81128,  34977,  21736,  85633, 124206, 168359,
       113048, 147342, 120014,  42154, 122299, 151739,  21943,  50434,
        13681,  65851, 110177]),
        20, 761],
       [3258, 16,
        array([218677,  42216,  74913,  74373,  63452,  62519,  54050,  62219,
        92001,  67647, 131492,  48164, 159796, 155154,  69934,  99725,
       161869,  66471,  45027]),
        20, 722],
       [19531, 8,
        array([224410,  35231, 150630,  54085,  38886, 120012,  76810,  90519,
        39687,  50451,  55995, 100131,  86860, 252626, 171089, 159056,
       131284,  56951, 160830]),
        20, 486]], dtype=object)

In [18]:
top_folder = '/data/workspace/yeqi/projects/RNN4REC/GRU4REC/Processed Data/min_5_max_20/top item above 20'
np.save(top_folder + '/X_train.npy', new_X_train)
np.save(top_folder + '/X_test.npy', new_X_test)